In [1]:
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)
from Doctopus.Utils.Tool import load_dict_from_pickle


dataset = "nba"
# load the dataset 
valipath = "C:/Users/jjli74/ljj/Doctopus-main/storedata/NBA.pkl"
val = load_dict_from_pickle(valipath)
print(val["total_token"])

Dictionary loaded from C:/Users/jjli74/ljj/Doctopus-main/storedata/NBA.pkl
27647


In [2]:
from Doctopus.var import datasets_attributes
from newDoc.get_current_rate import get_Strong_attris


folder_path = "C:/Users/jjli74/Desktop/nba/nba_sample"
attributes = datasets_attributes[dataset]

val_dict = val["validation"]
attribute_snippets = val_dict  # 请确保 res 已正确定义

cooc_matrix, markdown_table, attr_counts, total_chunks, associated_sets, fragment_top_k, attri_refs = get_Strong_attris(
    folder_path=folder_path,
    attribute_snippets=attribute_snippets,
    attributes=attributes,
    lang_model="en_core_web_sm",
    jaccard_threshold=0.5,
    k_clusters=3,
    top_k=3,
    bert_model="all-MiniLM-L6-v2",
    max_tokens_per_chunk=64
)

c:\Users\jjli74\AppData\Local\Continuum\anaconda3\envs\text2table\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


警告：属性集合 {'NBA_championships'} 的 Fragments 数量 1 小于 k=3，将 k 调整为 1
警告：属性集合 {'mvp_awards'} 的 Fragments 数量 1 小于 k=3，将 k 调整为 1
共现矩阵（NumPy 数组）：
[[ 0  9 10  9  3  3  1  1  6  0  0  0]
 [ 9  0  9  9  2  2  1  1  3  0  0  0]
 [10  9  0  9  2  2  1  1  4  0  0  0]
 [ 9  9  9  0  2  2  1  1  3  0  0  0]
 [ 3  2  2  2  0  0  0  0  0  0  0  0]
 [ 3  2  2  2  0  0  1  1  3  1  1  1]
 [ 1  1  1  1  0  1  0  5  1  0  0  1]
 [ 1  1  1  1  0  1  5  0  1  0  0  1]
 [ 6  3  4  3  0  3  1  1  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  1  1  0  0  0  0]]

共现矩阵（Markdown 表格）：
|                     |   name |   birth_date |   nationality |   age |   team |   position |   draft_pick |   draft_year |   college |   NBA_championships |   mvp_awards |   olympic_gold_medals |
|:--------------------|-------:|-------------:|--------------:|------:|-------:|-----------:|-------------:|-------------:|----------:|--------------------:|-------------:|-----

In [3]:
from newDoc.find_topk_chunks import get_topk_chunks


# Example inputs
folder_path = "C:\\Users\\jjli74\\Desktop\\nba\\nba"
attribute_dict = attri_refs
# Run the main function
result = get_topk_chunks(
    folder_path=folder_path,
    attribute_dict=attribute_dict,
    top_k=3,
    bert_model="all-MiniLM-L6-v2",
    lang_model="en_core_web_sm",
    max_tokens_per_chunk=64
)

print(result)



Top-K Chunks for Each Attribute in Each File:

File: C:\Users\jjli74\Desktop\nba\nba\Carrick_Felix.txt
  Attribute: name
    Top-1 Chunk: Carrick Felix Carrick Felix (born August 17, 1990) is an American former professional basketball player. He played three years of college basketball for Arizona State before being drafted by the Cleveland Cavaliers with the 33rd overall pick in the 2013 NBA draft. High school career Born in Las Vegas, Nevada, Felix played high school basketball at Millennium High School in Goodyear, Arizona. During his senior season at Millennium, Felix led his team to a state championship and was named MVP of the state championship.
    Top-2 Chunk: He was also named the Pac-12 Scholar Athlete of the Year and earned first-team All-Pac-12 Academic Team honors. Professional career Felix was selected with the 33rd overall pick in the 2013 NBA draft by the Cleveland Cavaliers. On August 20, 2013, he signed with the Cavaliers. During his rookie season, he had multiple a

In [4]:
from Doctopus.Utils.Tool import merge_chunks_to_set


result_final = merge_chunks_to_set(result)
print(result_final)




{'C:\\Users\\jjli74\\Desktop\\nba\\nba\\Carrick_Felix.txt': {"Felix was removed from Melbourne's active roster on February 16, 2018, following Prather's return from injury. He remained with the team for their championship run. In June 2018, Felix re-joined the Wizards for the 2018 NBA Summer League. NBA career statistics Regular season Personal In his two years away from the game of basketball between 2014 and 2016, Felix created a mobile app called ZGNIT, which helps holiday makers book adventure equipments such as jet skis.", 'He was also named the Pac-12 Scholar Athlete of the Year and earned first-team All-Pac-12 Academic Team honors. Professional career Felix was selected with the 33rd overall pick in the 2013 NBA draft by the Cleveland Cavaliers. On August 20, 2013, he signed with the Cavaliers. During his rookie season, he had multiple assignments to the Canton Charge of the NBA Development League.', 'Having "broke his left knee in half", Felix needed two surgeries and was told 

In [30]:

import json
import openai


prompt_template_for_extract = """
The content of the file is as follows:
{file_content}

Please extract the following attributes from the above content:
{attributes_list}

Return the result in JSON format, for example: {{"Attribute1": "Value1", "Attribute2": "Value2", ...}}.
If any attribute cannot be found, set its value to "".
    """

def clean_json_response(content):
    """
    清理模型返回的 JSON 响应，去掉多余的标记。
    
    Args:
        content (str): 从模型返回的字符串内容。
    
    Returns:
        dict: 如果解析成功，返回解析后的 JSON；否则返回错误消息。
    """
    try:
        # 去掉可能的 ```json 和 ``` 标记
        if content.startswith("```json"):
            content = content[7:]  # 去掉 ```json
        if content.endswith("```"):
            content = content[:-3]  # 去掉 ```
        
        # 去掉两端的多余空白
        content = content.strip()

        # 解析为 JSON 对象
        return json.loads(content)
    except json.JSONDecodeError as e:
        # 如果解析失败，返回错误信息

        print(content)
        return {"error": f"Failed to parse JSON response: {str(e)}"}
    
def extract_attributes_from_text(text,prompt_template,attribute_explanations,model="gpt-4o"):
    # if not text:
    try:
        
        prompt = prompt_template.format(
            file_content=text,
            attributes="\n".join(attribute_explanations)
        )

        client = openai.OpenAI(base_url="https://api.gptsapi.net/v1",
        api_key="sk-Hwv0998366e81e4a9a4cd0cd7f38890f3f7417d34134xAbD"
        )

        # Call OpenAI GPT model
        
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are an assistant for extracting information from documents."},
                {"role": "user", "content": prompt}
            ],
            temperature=0  # Ensure consistency
        )
        print(prompt)

        # Parse GPT response
        raw_content = response.choices[0].message.content.strip()
        print(raw_content)
        extracted_values = clean_json_response(raw_content)
        return extracted_values

    except Exception as e:
        return {"error": str(e)}
predict_res= {}
for filepath,text in result_final.items():
    attribute_value_dic = extract_attributes_from_text(text,prompt_template_for_extract, datasets_attributes[dataset] ,"gpt-4o-2024-08-06")
    predict_res[os.path.basename(filepath)] = attribute_value_dic
    print("1")
    break
print(predict_res)









1
{'Carrick_Felix.txt': {'error': "'attributes_list'"}}


In [25]:
from collections import Counter, defaultdict
import csv
import json
from textwrap import dedent
import numpy as np
import openai
import pandas as pd
from dataclasses import dataclass
from pydantic import BaseModel,Field
from typing import List
import json

def text_f1(preds=[], golds=[]):
    """Compute average F1 of text spans.
    Taken from Squad without prob threshold for no answer.
    """
    
    total_f1 = 0
    total_recall = 0
    total_prec = 0
    f1s = []
    if  not preds or not golds:
        return 0,0,0,0
    for pred, gold in zip(preds, golds):
        if isinstance(pred, list):
            pred = ' '.join(pred)  # Example way to convert list to string
        if isinstance(gold, list):
            gold = ' '.join(gold)  # Example way to convert list to string
        pred_toks = str(pred).split()
        gold_toks = str(gold).split()
        common = Counter(pred_toks) & Counter(gold_toks)
        num_same = sum(common.values())
        if len(gold_toks) == 0 or len(pred_toks) == 0:
            # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
            total_f1 += int(gold_toks == pred_toks)
            f1s.append(int(gold_toks == pred_toks))
        elif num_same == 0:
            total_f1 += 0
            f1s.append(0)
        else:
            precision = 1.0 * num_same / len(pred_toks)
            recall = 1.0 * num_same / len(gold_toks)
            f1 = (2 * precision * recall) / (precision + recall)
            total_f1 += f1
            total_recall += recall
            total_prec += precision
            f1s.append(f1)
    prec = total_prec / len(golds)
    recall =  total_recall/len(golds)
    f1_avg = total_f1 / len(golds)
    f1_median = np.percentile(f1s, 50)     
    return f1_avg,f1_median,prec,recall


def get_f1_dic(ground_truth,predict):

    all_file = list(predict.keys())
    # print(all_file)
    attributes = list(predict[all_file[0]].keys())

    data = defaultdict(lambda: defaultdict())

    for attribute in attributes:
        predict_list = []
        ground_list = []
        for file in all_file:
            try:
                p = str(predict[file][attribute])
                q = str(ground_truth[file][attribute])
                predict_list.append(p.lower())
                ground_list.append(q.lower())
            except :
                # print(f"{file} failed")
                pass
        if len(predict_list) ==0  or  len(ground_list) ==0:
            f1 = 0
            af1 = 0
            prec= 0
            recall = 0
        else:
            f1, af1,prec,recall = text_f1(predict_list,ground_list)

        data[attribute] = f1
    return data
       


    df = pd.DataFrame(data)

    # 反转表格
    df_transposed = df.set_index("Attribute").transpose()

    # 输出反转后的表格
    # print(df_transposed)
    return sum(data["avarag_F1_Score"])/len(data["avarag_F1_Score"])

gdfile = "C:\\Users\\jjli74\\Desktop\\nba\\NBA.json"

with open(gdfile, "r", encoding="utf-8") as file:
        ground = json.load(file)

get_f1_dic(ground,predict_res)







defaultdict(<function __main__.get_f1_dic.<locals>.<lambda>()>,
            {'name': 1.0,
             'birth_date': 0.8333333333333333,
             'nationality': 1.0,
             'age': 0.5,
             'team': 1.0,
             'position': 0.6166666666666666,
             'draft_pick': 1.0,
             'draft_year': 0.75,
             'college': 0.625,
             'NBA_championships': 0.25,
             'mvp_awards': 1.0,
             'olympic_gold_medals': 0.5,
             'FIBA_World_Cup': 0.5})